In [7]:
from glob import glob
import pandas as pd
# Find all directories under tennis_data/
date_folders = glob('tennis_data/*')

# Create an empty list to store all files
files_to_load_power = []

# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_to_load_power.extend(
        glob(f'{folder}/data/raw/raw_tennis_power_parquet/power*.parquet')
    )
df_power = [pd.read_parquet(file) for file in files_to_load_power]
df_power = pd.concat(df_power, ignore_index=True)

In [23]:
#match_away_team_info table
column_away= ['match_id','gender']
files_away= []
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_away.extend(
    [f for f in glob(f'{folder}/data/raw/raw_match_parquet/away_team*.parquet') if '_score' not in f]
    )
df_away_team_info = [pd.read_parquet(file, columns=column_away) for file in files_away]
df_away_team_info = pd.concat(df_away_team_info, ignore_index=True)

In [24]:
column_home= ['match_id','gender']

# Create an empty list to store all files
files_home= []
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_home.extend(
    [f for f in glob(f'{folder}/data/raw/raw_match_parquet/home_team*.parquet') if '_score' not in f]
    )
df_home_team_info = [pd.read_parquet(file, columns=column_home) for file in files_home]
df_home_team_info = pd.concat(df_home_team_info, ignore_index=True)

In [122]:
df_power.drop_duplicates(subset=['match_id', 'set_num', 'game_num'])

df = (
    df_power
    .merge(df_home_team_info[['match_id','gender']], on='match_id', how='left')
)

In [124]:
df.sample(5)

,match_id,set_num,game_num,value,break_occurred,gender
699434,12160977,2,8,10.00,False,F
642856,12110035,2,1,-54.90,False,F
507656,12128247,2,1,10.00,False,F
867682,12193293,2,2,36.80,True,M
603555,12147881,1,10,-28.64,True,M


In [127]:
# Remove duplicate games
unique_games = df_power.drop_duplicates(subset=['match_id', 'set_num', 'game_num'])

# Merge gender info from home team
match_gender = df_home_team_info[['match_id', 'gender']].drop_duplicates()
unique_games = unique_games.merge(match_gender, on='match_id', how='left')

# Count how many games per set
games_per_set = unique_games.groupby(['match_id', 'set_num', 'gender']).size().reset_index(name='games')

# Now compute average by gender
avg_games_by_gender = games_per_set.groupby('gender')['games'].mean().reset_index(name='avg_games_per_set')

print(avg_games_by_gender)


  gender  avg_games_per_set
0      F           8.938153
1      M           9.304300
